In [1]:
import requests 
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import duckdb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Data Scraping:

The original observed weather data sheet came in an a table format on the website, the very first step was to web scrape this table from the website and convert it into a dataframe.
We first scraped the table from the website and stored it in a dataframe called STID_df. This dataframe has stid (Airport code), latitude and the longitude.

In [2]:
#load the website
url = "https://forecast.weather.gov/stations.php"
result = requests.get(url)
if result.status_code != 200:
  print("something went wrong:", timeline_result.status_code, timeline_result.reason)
result

<Response [200]>

In [3]:
#extract the table 
source = result.text
soup=BeautifulSoup(source, 'html.parser')
result = soup.find_all("tr")
print(result)

[<tr><td align="right" height="19"><a href="http://www.weather.gov"><span class="nwslink"><b>weather.gov</b></span></a>   </td></tr>, <tr valign="top">
<td rowspan="2"><a href="http://www.noaa.gov"><img alt="NOAA logo - Click to go to the NOAA homepage" border="0" height="78" src="/weather/images/fcicons/noaaleft.jpg" width="85"/></a></td>
<td height="20"><img alt="Your National Weather Service" border="0" height="20" src="/images/wtf/forecast_blank.jpg" title="Your National Weather Service" width="500"/></td>
<td align="right" rowspan="2"><a href="http://weather.gov"><img alt="NWS logo" border="0" height="78" src="/weather/images/fcicons/nwsright.jpg" width="85"/></a></td>
<tr align="center">
<td background="/weather/images/fcicons/forecast_blank.jpg" class="white1" height="58">Observed Weather Stations</td>
</tr>
</tr>, <tr align="center">
<td background="/weather/images/fcicons/forecast_blank.jpg" class="white1" height="58">Observed Weather Stations</td>
</tr>, <tr>
<td><img alt="" 

From the cells below we can see that, we are looking at each row individually and adding the values to the colunms of STID, latitude and longitude to make it a dataframe.

In [4]:
parseString=result[6].text.strip() #start

print(parseString)

EB01  36.5-73.50


In [5]:
stid, latitude, longitude = [],[],[]

for event in result[6:]:
        #stid.append(event.find(lambda tag:tag.name=='td').text.strip())
        td_elements = event.find_all('td') 
        stid.append(td_elements[0].text.strip())
        latitude.append(td_elements[5].text.strip())
        longitude.append(td_elements[6].text.strip())

In [6]:
STID_df = pd.DataFrame({
    'STID': stid,
    'Latitude': latitude,
    'Longitude': longitude
})

Below is the output of the table we can see on the website. We can see that this table has 2193 rows, menaing that there are 2193 different airports in the table and 3 colunms. 

In [7]:
STID_df

,STID,Latitude,Longitude
0,EB01,36.5,-73.5
1,EB10,27.5,-88
2,EB31,27,-86
3,EB32,27,-84.3
4,EB33,58.5,-141
...,...,...,...
2134,TTCP,11.15,-60.833
2135,TTPP,10.617,-61.35
2136,TUPJ,18.45,-64.533
2137,TVSV,13.133,-61.2


Here we can see that we are web scraping a wikipedia page. This website has a table, which is the list of all the airports by the IATA airport code.This dataframe contains the IATA(name of the airports) and the ICAO(The ICAO airport code or location indicator is a four-letter code designating aerodromes around the world). In this case the ICAO would be for all the airports in the US

In [66]:
url2 = "https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_A"
result = requests.get(url2)
if result.status_code != 200:
  print("something went wrong:", timeline_result.status_code, timeline_result.reason)
result

source = result.text

soup=BeautifulSoup(source, 'html.parser')
result = soup.find_all("tr")
print(result)

[<tr>
<th style="text-align:left;"><a class="mw-redirect" href="/wiki/IATA" title="IATA">IATA</a>
</th>
<th style="text-align:left;"><a class="mw-redirect" href="/wiki/International_Civil_Aviation_Organization_airport_code" title="International Civil Aviation Organization airport code">ICAO</a>
</th>
<th style="text-align:left;">Airport name
</th>
<th style="text-align:left;">Location served
</th>
<th style="text-align:left;">Time
</th>
<th style="text-align:left;">DST
</th></tr>, <tr class="sortbottom">
<th colspan="6" style="text-align:left; background:#DDDDDD;"><span class="anchor" id="AA"></span>-AA-
</th></tr>, <tr>
<td>AAA</td>
<td>NTGA</td>
<td><a href="/wiki/Anaa_Airport" title="Anaa Airport">Anaa Airport</a></td>
<td><a href="/wiki/Anaa" title="Anaa">Anaa</a>, <a href="/wiki/Tuamotus" title="Tuamotus">Tuamotus</a>, <a href="/wiki/French_Polynesia" title="French Polynesia">French Polynesia</a></td>
<td><a href="/wiki/UTC%E2%88%9210:00" title="UTC−10:00">UTC−10:00</a></td>
<td>


In [9]:
iata, icao = [],[]
for event in result[2:555]:
        td_elements = event.find_all('td') 
        try:
            iata.append(td_elements[0].text.strip())
            icao.append(td_elements[1].text.strip())
        except: pass

Now we make a dataframe called DF2, this dataframe has the iata and icao values from the scraped wikipedia page

In [10]:
df2 = pd.DataFrame({
    'IATA':iata,
    'ICAO':icao
})
df2

,IATA,ICAO
0,AAA,NTGA
1,AAB,YARY
2,AAC,HEAR
3,AAD,HCAD
4,AAE,DABB
...,...,...
522,AZP,MMJC
523,AZR,DAUA
524,AZS,MDCY
525,AZT,


Now the airports_df is the dataframe that we use SQL to merge. We are joining the STID values from STID_df,(stid is similar to the airport code used by NOAA and NWS). This is joined with the ICAO values from DF2. We then select the STID, iata, latitude and longitude. We have used INNER JOIN for this as we want airports and STID codes that are in both the wikipedia and NWS and NOAA website. As INNER JOIN selects records that have matching values in both tables. 
This is why we now have 102 rows in our airports dataframe

In [11]:
airports_df=duckdb.sql('''
SELECT STID, iata,latitude,longitude
FROM STID_df
INNER JOIN df2 ON STID_df.stid=df2.icao
''').df()

In [12]:
airports_df

,STID,IATA,Latitude,Longitude
0,KABE,ABE,40.66,-75.43
1,KABQ,ABQ,35.05,-106.62
2,KABR,ABR,45.46,-98.43
3,KABY,ABY,31.54,-84.18
4,KACB,ACB,44.99,-85.2
...,...,...,...,...
97,PAWI,AIN,70.62,-159.86
98,PWAK,AWK,19.283,166.5
99,TAPA,ANU,17.117,-61.783
100,TNCA,AUA,12.5,-70.017


Now that we have the airports dataframe, we will make a list of all the STID values. 

In [13]:
stid_list = airports_df['STID'].tolist()
print(stid_list)

['KABE', 'KABQ', 'KABR', 'KABY', 'KACB', 'KACT', 'KACY', 'KADG', 'KADH', 'KADM', 'KADS', 'KAEL', 'KAEX', 'KAFF', 'KAFN', 'KAFW', 'KAGC', 'KAGS', 'KAHN', 'KAIA', 'KAID', 'KAIO', 'KAIZ', 'KAKO', 'KALB', 'KALI', 'KALM', 'KALN', 'KALO', 'KALS', 'KALW', 'KALX', 'KAMA', 'KAMN', 'KAMW', 'KANB', 'KAND', 'KANW', 'KAOH', 'KAOO', 'KAPA', 'KAPC', 'KAPG', 'KAPN', 'KARA', 'KARB', 'KARG', 'KARR', 'KART', 'KARV', 'KASE', 'KASH', 'KASL', 'KAST', 'KASX', 'KATL', 'KATY', 'KAUG', 'KAUM', 'KAUN', 'KAUO', 'KAUW', 'KAVP', 'KAVX', 'KAWM', 'KAXN', 'KAXS', 'KAYS', 'KAZO', 'KIWA', 'NSFA', 'PAAK', 'PADQ', 'PAGN', 'PAKN', 'PAKP', 'PANC', 'PANI', 'PANT', 'PANV', 'PARC', 'KAAF', 'KABI', 'KACK', 'KACV', 'KADW', 'KAKR', 'KAPF', 'KATS', 'KATW', 'KAUS', 'KAVL', 'KAXA', 'KAXX', 'PADK', 'PAFM', 'PATQ', 'PAWI', 'PWAK', 'TAPA', 'TNCA', 'TQPF']


Now I will Loop through the STID list values. 
For this we are using the national web servive weather API. This will help us access the data.
We first take the base that is the base website of the api.we then loop through all the station ID's and put it in the end f the api. If the status code is 200 then we print the json. If not then we output that the  

In [24]:
#base = "https://api.weather.gov/stations/"
#for value in stid_list:
    #end = f"{value}/observations"
    #r = requests.get(base + end)
    #if r.status_code == 200:
        #data = r.json()
    #else: 
        #print(error)

In [74]:
res = requests.get("https://api.weather.gov/stations/KABE/observations/2023-10-17T05:51:00+00:00")
res.json()

{'correlationId': '42475028',
 'parameterErrors': [{'parameter': 'path.time',
   'message': 'Invalid date-time "2023-9-16T05:51:00+00:00", expected format YYYY-MM-DDThh:mm:ssZ or YYYY-MM-DDThh:mm:ss+hh:mm'}],
 'title': 'Not Found',
 'type': 'https://api.weather.gov/problems/NotFound',
 'status': 404,
 'detail': 'Not Found',
 'instance': 'https://api.weather.gov/requests/42475028'}